In [1]:
using Revise

using RigidBodyDynamics
using RigidBodyDynamics: Bounds

using Plots
using BilevelTrajOpt

In [2]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)
body = findbody(mechanism, "ball")
basejoint = joint_to_parent(body, mechanism)
floatingjoint = Joint(basejoint.name, frame_before(basejoint), frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(mechanism, basejoint, floatingjoint)
position_bounds(floatingjoint) .= Bounds(-100, 100)
velocity_bounds(floatingjoint) .= Bounds(-100, 100)
μ = 0.5
motion_type = :xyz
point = Point3D(default_frame(findbody(mechanism, "floor")), [0.,0.,0.])
normal = FreeVector3D(default_frame(findbody(mechanism, "floor")), [0.,0.,1.])
hs = HalfSpace(point, normal)
floor = Obstacle(hs, μ, motion_type)
obstacles = [floor]
env = parse_contacts(mechanism, urdf, obstacles)
x0 = MechanismState(mechanism)
Δt = 0.005
sim_data = get_sim_data(x0,env,Δt,true);

In [15]:
q0 = [1., 0., 0., 0., 0., 0., 0.]
v0 = [0., 0., 0., 0.5, 0., 0.]

u0 = zeros(sim_data.num_v)
set_configuration!(x0, q0)
set_velocity!(x0, v0)
setdirty!(x0)
ctrl! = (u,t,x) -> u[:] = 0.
traj = BilevelTrajOpt.simulate(x0,env,sim_data.Δt,1,ctrl!,implicit_contact=false)
qnext = traj[1:sim_data.num_q,2]
vnext = traj[sim_data.num_q+1:sim_data.num_q+sim_data.num_v,2]
x_sol_exp = traj[sim_data.num_q+sim_data.num_v+2:end,2]

# qnext = q0
# vnext = v0

τ_ip, x_sol_ip, λ_ip, μ_ip, fx_ip = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=true);

τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

# display("Explicit")
# display(x_sol_exp)
display("IP")
display(x_sol_ip)
display("Aug Lag")
display(x_sol_auglag)

display("torques")
display(τ_ip)
display(τ_auglag)

elapsed time: 26.244335936 seconds


elapsed time: 0.00988763 seconds
elapsed time: 0.05556062 seconds
elapsed time: 0.017635784 seconds
elapsed time: 0.01527419 seconds
elapsed time: 0.02094907 seconds
elapsed time: 0.023038926 seconds
elapsed time: 0.014492373 seconds
elapsed time: 0.008789345 seconds
elapsed time: 0.025395749 seconds
elapsed time: 0.012417493 seconds
elapsed time: 0.009373214 seconds
elapsed time: 0.014666829 seconds
elapsed time: 0.010455644 seconds
elapsed time: 0.02786553 seconds
elapsed time: 0.017665234 seconds
elapsed time: 0.010005587 seconds
Solve_Succeeded
Solve_Succeeded

"IP"

6-element Array{Float64,1}:
 -9.65697e-9
 -9.80573e-9
  0.500003  
 -9.80573e-9
  0.475475  
  1.00001   

"Aug Lag"

6-element Array{Float64,1}:
  0.00318164
 -0.00318164
  0.503185  
 -0.00318164
  0.475478  
  1.00001   

"torques"

6-element Array{Float64,1}:
  0.0        
  0.0        
  0.0        
  4.90503    
 -6.00693e-16
 -9.81006    

6-element Array{Float64,1}:
  0.0       
  0.0       
  0.0       
  4.90503   
 -1.12062e-9
 -9.81006   

In [17]:
J_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2], vcat(qnext,vnext))

6×13 Array{Float64,2}:
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN

In [16]:
τ_auglag, x_sol_auglag, λ_auglag, μ_auglag, fx_auglag = solve_implicit_contact_τ(sim_data,q0,v0,u0,qnext,vnext,ip_method=false);

# autodiff 
J_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[2], vcat(qnext,vnext))
Jt_auto = ForwardDiff.jacobian(qv -> solve_implicit_contact_τ(sim_data,q0,v0,u0,qv[1:sim_data.num_q],qv[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)[1], vcat(qnext,vnext))

# numerical
ϵ = 1e-8
J_num = zeros(size(J_auto))
Jt_num = zeros(size(J_auto))
# Jt_num_ip = zeros(size(J_auto))
qv = vcat(qnext,vnext)
for i = 1:length(qv)
    δ = zeros(length(qv))
    δ[i] = ϵ
    τ, x, λ, μ, fx = solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=false)
    J_num[:,i] = (x - x_sol_auglag)/ϵ
    Jt_num[:,i] = (τ - τ_auglag)/ϵ
#     τ, x, λ, μ, fx = solve_implicit_contact_τ(sim_data,q0,v0,u0,(qv+δ)[1:sim_data.num_q],(qv+δ)[sim_data.num_q+1:sim_data.num_q+sim_data.num_v],ip_method=true)
#     Jt_num_ip[:,i] = (τ - τ_auglag)/ϵ
end

display(J_auto)
display(J_num)

display(Jt_auto)
display(Jt_num)
# display(Jt_num_ip)

6×13 Array{Float64,2}:
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN

6×13 Array{Float64,2}:
 0.0  -0.00321136   0.0621291     0.00157992   …   -0.00244326    1.32583    
 0.0  -0.000931057  0.184363     -0.491072         10.1866        3.77703    
 0.0  -0.00280909   0.0636965    -0.0253155        -0.0127195     1.30383    
 0.0   0.00568634   0.189654      0.512574        -10.1779        3.7832     
 0.0   0.000493128  0.000272332   0.000653871       0.000164324  -0.000261552
 0.0  -0.00131319   0.99885      -0.00566864   …   -0.0135064    20.3771     

6×13 Array{Float64,2}:
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN
 NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN

6×13 Array{Float64,2}:
 0.0    0.0           9.62965e-27  -1.03398e-17  …     0.0          0.0      
 0.0    0.0           0.0           0.0                0.0          0.0      
 0.0    1.03398e-17  -2.01948e-20   0.0                0.0          0.0      
 0.0    0.00394627   19.6355       -0.263844          -0.10081     -0.215883 
 0.0  -19.5552        0.0519104     0.0357012       -199.776        0.0605512
 0.0    0.0128827     0.0113447     0.0556094    …     0.132498  -199.899    

In [ ]:
display(Jt_auto[:,11:13])
display(Jt_num[:,11:13])
display(Jt_num_ip[:,11:13])

In [ ]:
 (J_num[:,1] - x_sol_auglag)/ϵ

In [ ]:
maximum(abs.(J_auto-J_num))

In [ ]:
abs.(J_auto-J_num)

In [ ]:
J_num

In [ ]:
J_auto